In [15]:
import numpy as np
import pandas as pd
import os
import math
import psycopg2
from sqlalchemy import create_engine

In [2]:
def lcSeq(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]

def lcStr(S,T):
    S = S.lower()
    T = T.lower()
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])
#     return lcs_set
    lc = [len(item) for item in list(lcs_set)]
    if len(lc) == 0:
        return 0
    else:
        return max(lc)

In [24]:
def team_match(names_a, names_b, source_a, source_b, method = 'subsequence'):
    team_names = pd.DataFrame()
    for a in names_a:
        arr = []
        if method == 'subsequence':
            for b in names_b:
                arr.append(lcSeq(a, b))
        else:
            for b in names_b:
                arr.append(lcStr(a, b))
        arr = pd.Series(arr)
        team_names = team_names.append({source_a: a, source_b: names_b[arr.idxmax()]}, ignore_index=True)
    return team_names

# def zipper(df1, df2, source_1, source_2):
#     match = team_match(df1, df2, source_1, source_2)
#     if match.duplicated(subset = source_1, keep = False).sum() + match.duplicated(subset = source_2, keep = False).sum() > 0:
#         match = team_match(df2.index.values, df1.index.values, source_2, source_1)
#         if match.duplicated(subset = source_1, keep = False).sum() + match.duplicated(subset = source_2, keep = False).sum() > 0:
#             match = team_match(df1.index.values, df2.index.values, source_1, source_2, 'substring')
#             if match.duplicated(subset = source_1, keep = False).sum() + match.duplicated(subset = source_2, keep = False).sum() > 0:
#                 match = team_match(df2.index.values, df1.index.values, source_2, source_1, 'substring')
#                 if match.duplicated(subset = source_1, keep = False).sum() + match.duplicated(subset = source_2, keep = False).sum() > 0:
#                     return 'Team Names Pileup'
#     df1 = match.set_index(source_1).join(df1)
#     df2 = match.set_index(source_2).join(df2)
#     df1 = df1.reset_index().set_index([source_1, source_2])
#     df2 = df2.reset_index().set_index([source_1, source_2])
#     return {source_1: df1, source_2: df2}

In [8]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [21]:
fte = fte_last_season(os.path.expanduser('~/Downloads/spi_matches.csv'), {'england':'2411'})
xpert = pd.read_csv(os.path.expanduser('~/Downloads/E0.csv'))
# fte

In [26]:
fte_names = fte['team1'].unique()
xpert_names = xpert['HomeTeam'].unique()

In [7]:
xpert_names

array(['Fulham', 'Crystal Palace', 'Liverpool', 'West Ham', 'West Brom',
       'Tottenham', 'Brighton', 'Sheffield United', 'Everton', 'Leeds',
       'Man United', 'Arsenal', 'Southampton', 'Newcastle', 'Chelsea',
       'Leicester', 'Aston Villa', 'Wolves', 'Burnley', 'Man City'],
      dtype=object)

In [27]:
team_match(xpert_names, fte_names, 'xpert', 'fte')

,fte,xpert
0,Fulham,Fulham
1,Crystal Palace,Crystal Palace
2,Liverpool,Liverpool
3,West Ham United,West Ham
4,West Bromwich Albion,West Brom
5,Tottenham Hotspur,Tottenham
6,Brighton and Hove Albion,Brighton
7,Sheffield United,Sheffield United
8,Everton,Everton
9,Leeds United,Leeds
